# Imports

In [1]:
from agent_arena_no_gathering import *
from environments.evil_wgw_env import EvilWindyGridWorld

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

### Agent training

In [3]:
num_actions = 4

In [ ]:
for i in range(5):
    for stoc in [0.0, 0.02, 0.05]:

        dqn_trainer = Trainer(
            num_actions,
            state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]],
            hiddens=[64, 64],
            huber_loss_delta=10.0
        )

        quant_trainer = Trainer(
            num_actions,
            state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]],
            hiddens=[64],
            num_atoms=64,
            distribution="quantile",
            huber_loss_delta=1.0
        )
        
        cat_trainer = Trainer(
            num_actions,
            state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]],
            hiddens=[64],
            num_atoms=64,
            values_range=(-1, 1),
            distribution="categorical"
        )

        trainers = [dqn_trainer, cat_trainer, quant_trainer]
        
        env = EvilWindyGridWorld(
            grid_size=(7, 10), visual=True, stochasticity=stoc)

        agent_arena = AgentArena(
            env, num_actions, trainers,
            save_path="results/031819/exp"+str(stoc)[2:]+"_"+str(i+1)
        )

        agent_arena.set_parameters(
            replay_memory_size=100000,
            discount_factor=0.99,
            max_episode_length=100
        )
    
        agent_arena.start(max_num_frames=2000000, batch_size=32)